# State Management with StateMachine
This notebook demonstrates how to work with state management using a StateMachine implementation. We'll explore how to create, manage, and control workflow states in a structured way.

## What we'll learn:
- Basic state machine concepts and implementation
- Creating and connecting workflow steps
- Managing state transitions and data flow
- Working with routing and loops in state machines
- Understanding state snapshots and execution flow

### Setup

In [1]:
from typing import TypedDict
from lib.state_machine import (
    StateMachine,
    Step,
    EntryPoint,
    Termination,
)

## Basic State Machine Concepts
Let's start with a simple example that demonstrates the core concepts of our state machine:
1. Defining state schema
2. Creating steps
3. Connecting steps
4. Running the workflow

**Creating the Schema and the State Machine**

In [ ]:
class Schema(TypedDict):
    """Schema defining the structure of our state.
    
    Attributes:
        input: The input value to process
        output: The processed output value
    """
    input: int
    output: int

In [ ]:
# Create our state machine instance
workflow = StateMachine(Schema)

**Defining the logic for Steps**

In [ ]:
def step_input(state: Schema) -> Schema:
    """First step: Increment the input value.
    
    Args:
        state: Current state containing input value
        
    Returns:
        Updated state with incremented value in output
    """
    return {"output": state["input"] + 1, "random": 10}



In [ ]:
def step_double(state: Schema) -> Schema:
    """Second step: Double the previous output.
    
    Args:
        state: Current state containing output from previous step
        
    Returns:
        Updated state with doubled output value
    """
    return {"output": state["output"] * 2}


**Creating and Connecting Steps**

In [5]:
entry = EntryPoint()
s1 = Step("input", step_input)
s2 = Step("double", step_double)
termination = Termination()

In [6]:
workflow.add_steps([entry, s1, s2, termination])

In [7]:
workflow.connect(entry, s1)
workflow.connect(s1, s2)
workflow.connect(s2, termination)

In [8]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['input'])],
 'input': [Transition('input' -> ['double'])],
 'double': [Transition('double' -> ['__termination__'])]}

**Running the Workflow**

In [9]:
initial_state = {"input": 4}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: input
[StateMachine] Executing step: double
[StateMachine] Terminating: __termination__


Run('0e3c6b52-8121-4378-a44e-49785aaa5888')

In [10]:
run_object.snapshots

[__entry__.Snapshot({'input': 4}),
 input.Snapshot({'input': 4, 'output': 5}),
 double.Snapshot({'input': 4, 'output': 10})]

## Advanced State Management: Routing and Loops
Now we'll explore more complex state management patterns including:
- Conditional routing between steps
- Creating loops in the workflow
- Managing state through multiple iterations

In [ ]:
class CounterSchema(TypedDict):
    """Schema for a counter-based workflow.
    
    Attributes:
        count: Current counter value
        max_value: Maximum value before termination
    """
    count: int
    max_value: int

In [12]:
workflow = StateMachine(CounterSchema)

In [ ]:
def increment_counter(state: CounterSchema) -> CounterSchema:
    """Increment the counter value.
    
    Args:
        state: Current state with counter value
        
    Returns:
        Updated state with incremented counter
    """
    return {"count": state["count"] + 1}

In [14]:
# Create steps
entry = EntryPoint()
increment = Step("increment", increment_counter)
termination = Termination()

In [15]:
workflow.add_steps([entry, increment, termination])

In [ ]:
# Router logic
def check_counter(state: CounterSchema) -> Step:
    """Determine next step based on counter value.
    
    Args:
        state: Current state with counter and max value
        
    Returns:
        Next step to execute (increment or terminate)
    """
    if state["count"] >= state["max_value"]:
        return termination
    return increment

In [17]:
# Connect steps with a loop in increment
workflow.connect(entry, increment)
workflow.connect(increment, [increment, termination], check_counter)

In [18]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['increment'])],
 'increment': [Transition('increment' -> ['increment', '__termination__'])]}

In [19]:
initial_state = {"count": 0, "max_value": 3}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Terminating: __termination__


Run('33630c81-9b65-42ce-b406-60523de03622')

In [20]:
run_object.snapshots

[__entry__.Snapshot({'count': 0, 'max_value': 3}),
 increment.Snapshot({'count': 1, 'max_value': 3}),
 increment.Snapshot({'count': 2, 'max_value': 3}),
 increment.Snapshot({'count': 3, 'max_value': 3})]

In [22]:
run_object.get_final_state()

{'count': 3, 'max_value': 3}